In [1]:
#opcua
from asyncua import Client, Node, ua
import asyncio
#arduino
import serial
import time
import math

Connection to OPC-UA

In [2]:
url = "opc.tcp://192.168.0.1:4840"
plc = Client(url)
await plc.connect()
n_rtd = plc.get_node("ns=3;s=\"PLC DB\".\"RTD\"")

Requested session timeout to be 3600000ms, got 30000ms instead


Connection to arduino via Serial

In [3]:
#initialize
arduino = serial.Serial(port='COM20', baudrate=115200, timeout=.1)

Simple single fan color writing

In [3]:
p11 = 4 #number of LEDs fully bright 0-9
p12 = 120 #brightness of the next LED 0-255
p21= 6 #number of LEDs fully bright 0-9
p22 = 200 #brightness of the next LED 0-255
color = 120 #hue value of displayed color (to be calculated from rtd reading with log scaling)

In [5]:
arduino.write(('<' + str(color) + ',' + str(p11) + ','  + str(p12) + ','  + str(p21) + ','  + str(p22) + '>').encode('ascii')) #240=>120

17

Constant update of fan color for 50 seconds

In [4]:
counter = 0
progress = 0
while counter < 100:
    rtd = await n_rtd.get_value()
    progress += 50
    p11 = int(progress/255)
    p12 = progress%255
    p21 = int((255*9 - progress)/255)
    p22 = (255*9 - progress)%255
    if (progress > 255*9):
        progress = 0
    color = int(min(max(math.log(rtd)-2.3,0)/3.2,1)*120+120)
    arduino.write(('<' + str(color) + ',' + str(p11) + ','  + str(p12) + ','  + str(p21) + ','  + str(p22) + '>').encode('ascii')) #240=>120
    time.sleep(0.5)
    counter = counter + 1

In [ ]:
arduino.write(('<' + str(color) + ',' + str(p11) + ','  + str(p12) + ','  + str(p21) + ','  + str(p22) + '>').encode('ascii')) #240=>120